<a href="https://colab.research.google.com/github/ErangaOttachchige/Final-Year-Research-Project/blob/main/02_stage1_detection_filter_training_OPTIMIZED_for_Colab_Free_T4_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Mount Drive + paths
from google.colab import drive
drive.mount("/content/drive")

import os
DRIVE_CCT = "/content/drive/MyDrive/datasets/cct20"
PROC_DIR = f"{DRIVE_CCT}/processed"
IMG_DIR  = f"{DRIVE_CCT}/eccv_18_all_images_sm"
print("Processed:", os.listdir(PROC_DIR))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed: ['cct20_species_annotations.csv', 'cct20_stage1_imagelevel.csv', 'cct20_stage2_species_imagelevel.csv', 'stage2_best_species_efficientnet_b0_optimized.pt', 'stage2_label_mapping.json']


In [8]:
# Install packages (once per runtime)
!pip -q install timm torchmetrics pandas numpy scikit-learn pillow tqdm


In [9]:
# Turn on GPU (in runtime settings) and check
import torch
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA: False
GPU: None


In [10]:
import pandas as pd
df1 = pd.read_csv("/content/drive/MyDrive/datasets/cct20/processed/cct20_stage1_imagelevel.csv")
print(df1["label_stage1"].value_counts())
print(df1["split"].value_counts())


label_stage1
animal    51237
empty      4014
car        2613
Name: count, dtype: int64
split
test_trans    23275
test_cis      15827
train         13553
val_cis        3484
val_trans      1725
Name: count, dtype: int64


In [11]:
# Mount Drive + paths + check files

from google.colab import drive
drive.mount("/content/drive")

import os
DRIVE_CCT = "/content/drive/MyDrive/datasets/cct20"
PROC_DIR  = f"{DRIVE_CCT}/processed"
CSV_STAGE1 = f"{PROC_DIR}/cct20_stage1_imagelevel.csv"

print("✓ PROC_DIR files:", os.listdir(PROC_DIR))
print("✓ Stage1 CSV exists:", os.path.exists(CSV_STAGE1))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ PROC_DIR files: ['cct20_species_annotations.csv', 'cct20_stage1_imagelevel.csv', 'cct20_stage2_species_imagelevel.csv', 'stage2_best_species_efficientnet_b0_optimized.pt', 'stage2_label_mapping.json']
✓ Stage1 CSV exists: True


In [ ]:
# Install + check GPU

!pip -q install timm torchmetrics pandas numpy scikit-learn pillow tqdm

import torch
print("✓ CUDA:", torch.cuda.is_available())
print("✓ GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


In [ ]:
# Load CSV + sanity checks
import pandas as pd, os

df = pd.read_csv(CSV_STAGE1)

missing = (~df["path"].apply(os.path.exists)).sum()
print(f"✓ Rows: {len(df)}, Missing paths: {missing}")
print("\nSplit counts:\n", df["split"].value_counts())
print("\nLabel counts:\n", df["label_stage1"].value_counts())

# Label mapping
classes = ["empty", "animal", "car"]  # fixed order
class_to_idx = {c:i for i,c in enumerate(classes)}
idx_to_class = {i:c for c,i in class_to_idx.items()}
df["y"] = df["label_stage1"].map(class_to_idx)

df.head()


In [ ]:
# OPTIONAL speed boost (recommended): cache resized tensors locally

# This avoids slow Drive reads and prevents crashes.
import os, glob, hashlib
import torch
from PIL import Image
import torchvision.transforms as T
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from functools import partial

CACHE_DIR = "/content/stage1_cache_224"
os.makedirs(CACHE_DIR, exist_ok=True)

def cache_path(img_path):
    return os.path.join(CACHE_DIR, hashlib.md5(img_path.encode()).hexdigest() + ".pt")

df["cache_path"] = df["path"].apply(cache_path)

cached = len(glob.glob(CACHE_DIR + "/*.pt"))
print("Cached tensors now:", cached, " / ", len(df))

pre_tf = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
])

def process_one(row):
    cp = row["cache_path"]
    if os.path.exists(cp):
        return
    try:
        img = Image.open(row["path"]).convert("RGB")
        x = pre_tf(img)
        torch.save(x, cp)
    except Exception as e:
        # print and skip bad files
        print("Error:", row["path"], e)

if cached < len(df)*0.95:
    print("🔄 Caching tensors locally (one-time per session)...")
    rows = [r for _, r in df.iterrows()]
    with ThreadPoolExecutor(max_workers=8) as ex:
        list(tqdm(ex.map(process_one, rows), total=len(rows), desc="Caching Stage1"))
    print("✓ Cache done:", len(glob.glob(CACHE_DIR + "/*.pt")))
else:
    print("✓ Cache already exists — skipping")


In [ ]:
# Dataset + Dataloaders (balanced sampling + stable)
import numpy as np
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import torch

# splits
train_df = df[df["split"]=="train"].reset_index(drop=True)
val_df   = df[df["split"]=="val_cis"].reset_index(drop=True)
valT_df  = df[df["split"]=="val_trans"].reset_index(drop=True)
test_cis_df   = df[df["split"]=="test_cis"].reset_index(drop=True)
test_trans_df = df[df["split"]=="test_trans"].reset_index(drop=True)

print("train:", len(train_df), "val_cis:", len(val_df), "val_trans:", len(valT_df),
      "test_cis:", len(test_cis_df), "test_trans:", len(test_trans_df))

# augment on cached tensors (works)
aug_tf = T.Compose([
    T.RandomHorizontalFlip(0.5),
    T.ColorJitter(0.2, 0.2, 0.1),
])

class CachedDS(Dataset):
    def __init__(self, frame, augment=False):
        self.df = frame.reset_index(drop=True)
        self.augment = augment
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        x = torch.load(r["cache_path"])          # fast
        if self.augment:
            x = aug_tf(x)
        y = int(r["y"])
        return x, y

train_ds = CachedDS(train_df, augment=True)
val_ds   = CachedDS(val_df, augment=False)
valT_ds  = CachedDS(valT_df, augment=False)
test_cis_ds   = CachedDS(test_cis_df, augment=False)
test_trans_ds = CachedDS(test_trans_df, augment=False)

# balanced sampling on train
counts = train_df["y"].value_counts().sort_index()
w_class = 1.0 / counts
w_sample = train_df["y"].map(w_class).values
sampler = WeightedRandomSampler(torch.tensor(w_sample, dtype=torch.double),
                                num_samples=len(w_sample), replacement=True)

# class weights (loss)
cw = (counts.sum() / (len(counts) * counts)).values
class_weight = torch.tensor(cw, dtype=torch.float32)

print("Train class counts:\n", counts)
print("Loss class weights:\n", class_weight)

# loaders
device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 2 if device=="cuda" else 0
PIN = True if device=="cuda" else False

BATCH_TRAIN = 32
BATCH_EVAL  = 64

train_loader = DataLoader(train_ds, batch_size=BATCH_TRAIN, sampler=sampler,
                          num_workers=NUM_WORKERS, pin_memory=PIN)
val_loader   = DataLoader(val_ds, batch_size=BATCH_EVAL, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=PIN)
valT_loader  = DataLoader(valT_ds, batch_size=BATCH_EVAL, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=PIN)
test_cis_loader   = DataLoader(test_cis_ds, batch_size=BATCH_EVAL, shuffle=False,
                               num_workers=NUM_WORKERS, pin_memory=PIN)
test_trans_loader = DataLoader(test_trans_ds, batch_size=BATCH_EVAL, shuffle=False,
                               num_workers=NUM_WORKERS, pin_memory=PIN)


In [ ]:
# Train EfficientNet-B0 (AMP + accumulation + saves best)
import timm
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✓ Device:", device)

model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=len(classes)).to(device)
crit  = nn.CrossEntropyLoss(weight=class_weight.to(device))
opt   = torch.optim.AdamW(model.parameters(), lr=3e-4)

scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

ACCUM_STEPS = 2   # effective batch = 32 * 2 = 64

def eval_loader(loader, name="eval"):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x,y in tqdm(loader, desc=name, leave=True):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=(device=="cuda")):
                logits = model(x)
            p = logits.argmax(1)
            ys.extend(y.cpu().tolist())
            ps.extend(p.cpu().tolist())
    acc = accuracy_score(ys, ps)
    mf1 = f1_score(ys, ps, average="macro")
    return acc, mf1, ys, ps

SAVE_PATH = f"{PROC_DIR}/stage1_best_empty_animal_car.pt"
best = -1

EPOCHS = 5
for ep in range(1, EPOCHS+1):
    model.train()
    opt.zero_grad(set_to_none=True)
    running = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {ep}/{EPOCHS} [train]", leave=True)
    for i, (x,y) in enumerate(pbar, 1):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            logits = model(x)
            loss = crit(logits, y) / ACCUM_STEPS

        scaler.scale(loss).backward()

        if i % ACCUM_STEPS == 0:
            scaler.step(opt)
            scaler.update()
            opt.zero_grad(set_to_none=True)

        running += loss.item() * x.size(0) * ACCUM_STEPS
        pbar.set_postfix({"loss": float(loss.item()*ACCUM_STEPS)})

    train_loss = running / len(train_df)

    val_acc, val_mf1, _, _ = eval_loader(val_loader, "val_cis")
    print(f"\nEpoch {ep}: train_loss={train_loss:.4f} | val_cis acc={val_acc:.3f} macroF1={val_mf1:.3f}")

    if len(valT_df) > 0:
        vt_acc, vt_mf1, _, _ = eval_loader(valT_loader, "val_trans")
        print(f"          val_trans acc={vt_acc:.3f} macroF1={vt_mf1:.3f}")

    if val_mf1 > best:
        best = val_mf1
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"💾 SAVED BEST: {SAVE_PATH} (macroF1={best:.3f})")


In [ ]:
# Final evaluation on CIS test + TRANS test
from sklearn.metrics import classification_report

print("\n" + "="*60)
print("FINAL TEST EVALUATION (Stage 1)")
print("="*60)

model.load_state_dict(torch.load(SAVE_PATH, map_location=device))

cis_acc, cis_mf1, cis_y, cis_p = eval_loader(test_cis_loader, "test_cis")
tr_acc,  tr_mf1,  tr_y,  tr_p  = eval_loader(test_trans_loader, "test_trans")

print(f"\n🎯 TEST CIS   → acc={cis_acc:.3f}, macroF1={cis_mf1:.3f}")
print(f"🎯 TEST TRANS → acc={tr_acc:.3f}, macroF1={tr_mf1:.3f}")

print("\n--- CIS REPORT ---")
print(classification_report(cis_y, cis_p, target_names=[idx_to_class[i] for i in range(len(classes))]))

print("\n--- TRANS REPORT ---")
print(classification_report(tr_y, tr_p, target_names=[idx_to_class[i] for i in range(len(classes))]))


In [ ]:
# Save label mapping (for inference pipeline)
import json

mapping = {
    "classes": classes,
    "class_to_idx": class_to_idx
}
out_json = f"{PROC_DIR}/stage1_label_mapping.json"
with open(out_json, "w") as f:
    json.dump(mapping, f, indent=2)

print("✓ Saved:", out_json)
print("✓ Stage 1 done!")
